In [ ]:
text = "Sau hai thế kỷ, chính sách đóng cửa đất nước dưới thời Mạc phủ Tokugawa đã đi đến kết thúc khi Nhật Bản bị Hoa Kỳ ép mở cửa giao thương vào năm 1854. Những năm tiếp theo cuộc Minh Trị duy tân năm 1868 và sự sụp đổ của chế độ mạc phủ, Nhật Bản đã tự chuyển đổi từ một xã hội khá lạc hậu và phong kiến sang một quốc gia công nghiệp hiện đại. Nhật đã cử các phái đoàn và sinh viên đi khắp thế giới để học và tiếp thu khoa học và nghệ thuật phương Tây, điều này đã được thực hiện nhằm giúp Nhật Bản tránh khỏi rơi vào ách thống trị của nước ngoài và cũng giúp cho Nhật có thể cạnh tranh ngang ngửa với các cường quốc phương Tây.",

# INIT VNCORENLP MODEL

In [2]:
import py_vncorenlp
from word_lists import PEOPLE, PLACE, ORGANIZATION, number_map
# Automatically download VnCoreNLP components from the original repository
py_vncorenlp.download_model(save_dir=r'D:\Study\DATN\model\NKD-GNN-test\VnCoreNLP')

# Load VnCoreNLP from the local working folder that contains both `VnCoreNLP-1.2.jar` and `models`
model_vncore = py_vncorenlp.VnCoreNLP(annotators=["wseg", "pos", "ner"], save_dir=r'D:\Study\DATN\model\NKD-GNN-test\VnCoreNLP')

VnCoreNLP model folder D:\Study\DATN\model\NKD-GNN-test\VnCoreNLP already exists! Please load VnCoreNLP from this folder!


In [ ]:
def generate_template_caption(text, model_vncore):
    PERSON_PLACEHOLDER = "<PERSON>"
    PLACE_PLACEHOLDER = "<PLACE>"
    ORGANIZATION_PLACEHOLDER = "<ORGANIZATION>"
    # Split into sentences
    sentences = text.split(".")

    # Annotate the chunk with VnCoreNLP
    caption = []
    for splited_sentence in sentences:
        annotations = model_vncore.annotate_text(splited_sentence)

        # Initialize a list to store the processed tokens
        result = []

        # Variables to handle multi-token entities
        current_entity = None
        current_placeholder = None

        # Loop over each sentence in the annotation result (annotations is a dictionary with indices as keys)
        for sentence_key in annotations:
            sentence = annotations[sentence_key]
            #To reduce from 2-3 posTag "N" sit next together only return 1 "N" tag
            flag = False
            # Process each token in the sentence
            prev_word = ""
            prev_pos_tag = ""
            for token in sentence:
                word = token['wordForm']
                pos_tag = token['posTag']
                if pos_tag == 'N' and flag == False:
                    if word.lower() in PEOPLE:
                        if prev_pos_tag == 'M':
                            try:
                                num_person = number_map[prev_word.lower()]
                            except:
                                num_person = 1
                            result.pop()
                            for i in range(num_person - 1):
                                result.append(PERSON_PLACEHOLDER + ",")
                        result.append(PERSON_PLACEHOLDER)
                    elif word.lower() in PLACE:
                        result.append(PLACE_PLACEHOLDER)
                    elif word.lower() in ORGANIZATION:
                        result.append(ORGANIZATION_PLACEHOLDER)
                    else:
                        result.append(word)
                    flag = True
                elif pos_tag == 'N' and flag == True:
                    continue
                elif pos_tag !='Nc' and pos_tag != 'P':
                    result.append(word)
                    flag = False
                    prev_word = word
                    prev_pos_tag = pos_tag

        caption.append(' '.join(result))
    # Join the processed tokens back into a sentence
    return '. '.join(caption).strip(" .")

In [7]:
text = "Sau hai thế kỷ, chính sách đóng cửa đất nước dưới thời Mạc phủ Tokugawa đã đi đến kết thúc khi Nhật Bản bị Hoa Kỳ ép mở cửa giao thương vào năm 1854. Những năm tiếp theo cuộc Minh Trị duy tân năm 1868 và sự sụp đổ của chế độ mạc phủ, Nhật Bản đã tự chuyển đổi từ một xã hội khá lạc hậu và phong kiến sang một quốc gia công nghiệp hiện đại. Nhật đã cử các phái đoàn và sinh viên đi khắp thế giới để học và tiếp thu khoa học và nghệ thuật phương Tây, điều này đã được thực hiện nhằm giúp Nhật Bản tránh khỏi rơi vào ách thống trị của nước ngoài và cũng giúp cho Nhật có thể cạnh tranh ngang ngửa với các cường quốc phương Tây.",

In [9]:
test_str = "Nguyễn Sinh Phúc và Nguyễn Phúc Trọng đang bắt tay nhau tại Tòa nhà Quốc hội. Hai người đàn ông đang đánh nhau ở quảng trường"

In [14]:
print(generate_template_caption(text = test_str, model_vncore=model_vncore))

Nguyễn Sinh Phúc và Nguyễn Phúc Trọng đang bắt tay nhau tại Tòa nhà Quốc hội. Hai người đàn ông đang đánh nhau ở quảng trường
Nguyễn_Sinh_Phúc và Nguyễn_Phúc_Trọng đang bắt_tay nhau tại <PLACE>. <PERSON>, <PERSON> đang đánh nhau ở <ORGANIZATION>


# RAW CAPTION GENERATE

In [18]:
!pip install gensim

  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/f5/57/f2e6568dbf464a4b270954e5fa3dee4a4054d163a41c0e7bf0a34eb40f0f/gensim-4.3.3-cp311-cp311-win_amd64.whl.metadata
  Using cached gensim-4.3.3-cp311-cp311-win_amd64.whl.metadata (8.2 kB)
  Obtaining dependency information for scipy<1.14.0,>=1.7.0 from https://files.pythonhosted.org/packages/4a/48/4513a1a5623a23e95f94abd675ed91cfb19989c58e9f6f7d03990f6caf3d/scipy-1.13.1-cp311-cp311-win_amd64.whl.metadata
  Using cached scipy-1.13.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Obtaining dependency information for smart-open>=1.8.1 from https://files.pythonhosted.org/packages/06/bc/706838af28a542458bffe74a5d0772ca7f207b5495cd9fccfce61ef71f2a/smart_open-7.0.5-py3-none-any.whl.metadata
  Using cached smart_open-7.0.5-py3-none-any.whl.metadata (24 kB)
  Obtaining dependency information for wrapt from https://files.pythonhosted.org/packages/63/bb/c293a67fb765a2ada48f48cd0f2bb957da8161439da4c03ea123b9894


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import torch
import gensim

def load_pretrained_embeddings(vocab, embedding_dim, embedding_file):
    """
    Load pretrained embeddings and create a weights matrix.

    Parameters:
    - vocab: Dictionary mapping words to indices in the vocabulary
    - embedding_dim: Dimension of the embeddings
    - embedding_file: Path to the pretrained embedding file (e.g., .vec or .txt)

    Returns:
    - weights_matrix: A matrix of shape (vocab_size, embedding_dim)
    """
    # Load pretrained embeddings using gensim
    embeddings = gensim.models.KeyedVectors.load_word2vec_format(embedding_file, binary=False)

    vocab_size = len(vocab)
    weights_matrix = torch.zeros((vocab_size, embedding_dim))

    # Fill weights matrix with pretrained embeddings
    for word, idx in vocab.items():
        if word in embeddings:
            weights_matrix[idx] = torch.tensor(embeddings[word])
        else:
            # Randomly initialize missing words
            weights_matrix[idx] = torch.randn(embedding_dim)

    return weights_matrix


In [ ]:
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torchvision.models as models
from torchvision import transforms
from PIL import Image
from preprocess_image import resize_image





######################## TEMPLATE CAPTION GENERATOR ###############################

# CNN Encoder with Pre-trained
class VGG19Encoder(nn.Module):
    def __init__(self, embed_size):
        super(VGG19Encoder, self).__init__()
        # Load pre-trained
        vgg = models.vgg19(pretrained=True)
        # Remove the last fully connected layer (fc) of 
        self.vgg = nn.Sequential(*list(vgg.children())[:-1])
        # Linear layer to map ResNet output to embedding size
        self.fc = nn.Linear(vgg.fc.in_features, embed_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, images):
        # Extract features using 
        with torch.no_grad():  # Disable gradient computation for ResNet
            features = self.vgg(images)
        # Flatten the features and pass through a linear layer
        features = features.view(features.size(0), -1)
        features = self.fc(features)
        features = self.relu(features)
        features = self.dropout(features)
        return features
    
class LSTMDecoder(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1, pretrained_embeddings=None):
        super(LSTMDecoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)

        # Load pretrained embeddings if provided
        if pretrained_embeddings is not None:
            self.embedding.weight.data.copy_(pretrained_embeddings)
            self.embedding.weight.requires_grad = False  # Freeze embeddings if desired

        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, features, captions):
        embeddings = self.embedding(captions)
        embeddings = torch.cat((features.unsqueeze(1), embeddings), dim=1)
        hiddens, _ = self.lstm(embeddings)
        outputs = self.fc(hiddens)
        return outputs
    
# Full model combining CNN encoder and LSTM decoder
class ImageCaptioningModel(nn.Module):
    def __init__(self, embed_size=256, hidden_size=512, vocab_size=10000, num_layers=1, embedding = None):
        super().__init__()
        self.encoder = VGG19Encoder(embed_size)
        self.decoder = LSTMDecoder(embed_size, hidden_size, vocab_size, num_layers,embedding)

    def forward(self, images, embedding):
        features = self.encoder(images)
        outputs = self.decoder(features, embedding)
        return outputs

    def generate_caption(self, image, max_length=20, vocab=None):
        # Preprocess the image
        image = resize_image(image)
        features = self.encoder(image)

        # Start decoding
        captions = [vocab['<START>']]  # Begin with the start token
        for _ in range(max_length):
            caption_tensor = torch.tensor(captions).unsqueeze(0).to(features.device)
            outputs = self.decoder(features, caption_tensor)
            next_word_id = outputs.argmax(2)[:, -1].item()  # Get the predicted next word
            captions.append(next_word_id)
            if vocab and next_word_id == vocab['<END>']:
                break
        return captions

In [ ]:
# Example vocabulary
vocab = {'<PAD>': 0, '<START>': 1, '<END>': 2, 'tôi': 3, 'đi': 4, 'học': 5}  # Example vocab
embedding_dim = 300
embedding_file = "cc.vi.300.vec"  # Path to FastText Vietnamese embeddings

pretrained_weights = load_pretrained_embeddings(vocab, embedding_dim, embedding_file)


# Instantiate the model
model = ImageCaptioningModel(embed_size=256, hidden_size=512, vocab_size=len(vocab))

# Load an image and generate a caption
image_path = "path/to/image.jpg"
caption_ids = model.generate_caption(image_path, vocab=vocab)

# Convert caption IDs to words
reverse_vocab = {v: k for k, v in vocab.items()}
caption = ' '.join(reverse_vocab[id] for id in caption_ids if id in reverse_vocab)
print("Generated Caption:", caption)
